In [3]:
import duckdb
import pandas as pd
import os
import glob
import re

DATA_DIR = "Sales Dataset"
DB_PATH = "salesdata.duckdb"

import pandas as pd

def clean_df(df):
    # --- 1. Drop fully empty rows ---
    df = df.replace(r'^\s*$', pd.NA, regex=True)
    df = df.dropna(how='all')

    # --- 2. Remove index-like columns ---
    index_like = ["index", "idx", "id", "row"]

    df = df.drop(
        columns=[c for c in df.columns if c.strip().lower() in index_like],
        errors="ignore"
    )

    # --- 3. Identify unnamed columns ---
    unnamed_cols = [col for col in df.columns if "unnamed" in col.lower()]

    if not unnamed_cols:
        return df.reset_index(drop=True)

    # --- 4. Read row 0 for potential header names ---
    first_row = df.iloc[0]

    rename_map = {}
    for col in unnamed_cols:
        val = first_row[col]

        # Valid header must be a non-empty string
        if isinstance(val, str) and val.strip():
            rename_map[col] = val.strip()

    # If nothing to rename → do NOT drop first row
    if not rename_map:
        return df.reset_index(drop=True)

    # --- 5. Apply renaming ---
    df = df.rename(columns=rename_map)

    # --- 6. Drop the first row (it contained header values) ---
    df = df.iloc[1:].reset_index(drop=True)

    return df

def clean_table_name(name):
    """Convert filename into a safe SQL table name."""
    name = name.lower()
    name = re.sub(r'[^a-z0-9]+', '_', name)   # replace spaces/special chars with _
    return name.strip('_')


def clean_column(col):
    return (
        col.strip()
        .lower()
        .replace(" ", "_")
        .replace("-", "_")
        .replace("/", "_")
        .replace(".", "_")
    )

def load_csv(path):
    return pd.read_csv(path)


def create_duckdb_database():
    print("📌 Initializing DuckDB...")

    con = duckdb.connect(DB_PATH)

    csv_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))

    if not csv_files:
        print("❌ No CSV files found in /data")
        return

    print(f"📁 Found {len(csv_files)} CSV files")

    for csv_path in csv_files:

        filename = os.path.basename(csv_path)
        table_name = clean_table_name(os.path.splitext(filename)[0])

        print(f"\n📌 Processing file: {filename}")
        print(f"➡ Table name will be: {table_name}")

        df = load_csv(csv_path)

        df.columns = [clean_column(c) for c in df.columns]
        df = clean_df(df)
        # Use safe quoted identifiers
        con.execute(f'DROP TABLE IF EXISTS "{table_name}";')
        con.execute(f'CREATE TABLE "{table_name}" AS SELECT * FROM df;')

        print(f"✔ Loaded {len(df)} rows into table {table_name}")

    print("\n🔍 DB Tables:")
    print(con.execute("SHOW TABLES;").fetchdf())

    con.close()
    print("\n✅ DuckDB created successfully!")


create_duckdb_database()


📌 Initializing DuckDB...
📁 Found 7 CSV files

📌 Processing file: Amazon Sale Report.csv
➡ Table name will be: amazon_sale_report


/var/folders/w8/1yvv2gxn78569_4jrzjkjb8c0000gn/T/ipykernel_25819/2409023858.py:72: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)


✔ Loaded 128975 rows into table amazon_sale_report

📌 Processing file: Cloud Warehouse Compersion Chart.csv
➡ Table name will be: cloud_warehouse_compersion_chart
✔ Loaded 49 rows into table cloud_warehouse_compersion_chart

📌 Processing file: P  L March 2021.csv
➡ Table name will be: p_l_march_2021
✔ Loaded 1330 rows into table p_l_march_2021

📌 Processing file: May-2022.csv
➡ Table name will be: may_2022
✔ Loaded 1330 rows into table may_2022

📌 Processing file: Sale Report.csv
➡ Table name will be: sale_report
✔ Loaded 9271 rows into table sale_report

📌 Processing file: International sale Report.csv
➡ Table name will be: international_sale_report
✔ Loaded 37432 rows into table international_sale_report

📌 Processing file: Expense IIGF.csv
➡ Table name will be: expense_iigf
✔ Loaded 16 rows into table expense_iigf

🔍 DB Tables:
                               name
0                amazon_sale_report
1  cloud_warehouse_compersion_chart
2                      expense_iigf
3         int